In [1]:
import os
import re
import time
import json
import datetime as dt
import pandas as pd
from tqdm import tqdm

# from sqlalchemy import event, create_engine

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.action_chains import ActionChains

# 抓取目標看板

In [2]:
import requests
r = requests.get("https://www.dcard.tw/service/api/v2/forums?nsfw=true&withPostListExcludeForums=false")
print(r.status_code)

200


In [3]:
# 列出所有看板
topic_list = r.json()
# print(type(r))
# for i in topic_list:
#     print(i["name"])

# # 隱藏看板
# df.invisible.unique()
# df.query("invisible == True").name.unique()

# 表格化
df_forums  = pd.DataFrame(topic_list)
# df_forums.head()

# 挑出重點
df_forums_1 = df_forums[["id","alias","name","subscriptionCount","createdAt","updatedAt","postCount",]]
# df_forums_1.head()

# 重新排序
def get_v(c):
    postCount_last30Days = (c['last30Days'])
    
    return postCount_last30Days

df_forums_1["postCount_last30Days"] = df_forums_1.postCount.apply(get_v)

# 追蹤數最多的版
most_followers = df_forums_1.sort_values(['subscriptionCount'], ascending = False).reset_index(drop=True)
most_followers_top20 = most_followers.head(20)
# most_followers_top20

# 一個月內發文數最多的版
most_post = df_forums_1.sort_values(['postCount_last30Days'], ascending = False).reset_index(drop=True)
most_post_top20 = most_post.head(20)
# most_post_top20

df_forums_final = pd.concat([most_followers_top20, most_post_top20]).reset_index(drop = True)
# pd.concat([most_followers, most_post]).name.head(30)

# 最終挑選的看板
df_forums_final.alias.unique()
# len(df_forums_final.alias.unique())

/var/folders/1y/scrkjqxx3_z6b3c7zh3v1wk00000gn/T/ipykernel_2586/3424270636.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forums_1["postCount_last30Days"] = df_forums_1.postCount.apply(get_v)


array(['dressup', 'meme', 'sex', 'relationship', 'talk', 'makeup', 'food',
       'youtuber', 'money', 'horoscopes', 'weight_loss', 'trending',
       'netflix', 'pet', 'job', 'stock', 'girl', 'mood', 'joke', 'funny',
       'whysoserious', 'aov', 'mole_world', 'rainbow', 'freshman', 'exam',
       'savemoney', '3c'], dtype=object)

# 登入動作及控制

In [20]:
# Selenium模擬瀏覽器
drivePath = './chromedriver'
browser = webdriver.Chrome(drivePath)

/var/folders/1y/scrkjqxx3_z6b3c7zh3v1wk00000gn/T/ipykernel_2586/3456079939.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(drivePath)


In [21]:
def quick_start(mode="login"):
    # 1 啟動 Selenium模擬瀏覽器
    drivePath = './chromedriver'
    browser = webdriver.Chrome(drivePath)

    # 2 開啟首頁
    browser.get('https://www.dcard.tw')    
    time.sleep(2)
        
    # 3  登入
    if mode == 'login':
        userEmail = ""
        userPassword = ""

        login(browser, userEmail, userPassword)
    
    return browser

def login(browser, userEmail, userPassword):
    # 進入登入畫面
    browser.find_element_by_css_selector(".bKXiml a").click()

    time.sleep(3)    

    try:
        # 填寫登入信箱與密碼
        ele_input_email = browser.find_element_by_css_selector("#email")
        ele_input_password = browser.find_element_by_css_selector("#password")
        ele_button_send = browser.find_element_by_css_selector("button.rSTIv")

        ele_input_email.send_keys("bb20020116@gmail.com")
        time.sleep(0.3)
        ele_input_password.send_keys("Brian20020116!!")
        time.sleep(0.3)
        ele_button_send.click()

        print("> Login Sucessfully.")

    except Exception as e:
        print(f"> Login Failed: {e}")

# 抓取看板內的文章

In [22]:
def fetch_forumArticles(forumName, epoch=30):
    ''' 爬取單一看板上，最新的文章，並以 epoch作為向下移動的次數 '''
    # 1 前往目標頁面
    firstPageUrl = f"https://www.dcard.tw/f/{forumName}?latest=true"
    browser.get(firstPageUrl)

    # 2 Scrape all
    allDfList = []

    for i in range(epoch):
        if i % 10 == 0:
            print(f">> current: {i}/{epoch}")

        # 爬現在的文章
        df = fetch_currentArticles(browser)
        allDfList.append(df)

        # 往下移動
        ActionChains(browser).send_keys(Keys.END).perform()

        time.sleep(0.3)

    # 3 去重複
    allDf_origin = pd.concat(allDfList)
    allDf = allDf_origin.drop_duplicates("Link")
    allDf = allDf.dropna(subset=['Name']).reset_index(drop=True)

    # 4 Set forum name
    allDf["ForumName"] = forumName
    allDf.drop(columns=["Name"], inplace=True)

    return allDf

def fetch_currentArticles(browser):
    html = browser.page_source
    soup = BeautifulSoup(html)
    articles = soup.select("article.hDasRN ")

    artList = []

    for art in articles:
        artTitle = art.find('a').text
        artHref = art.find('a').get("href")
        artCollege = art.find('div', class_='pKOgj')
        artName = art.find('span', class_='dURHrx')
        artDict = {"Title": artTitle, "Link": artHref, "Name": artName}
        artList.append(artDict)

    df = pd.DataFrame(artList)

    return df

In [23]:
%%time

# 此次設定
epoch = 30
startTimeStr = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M") # 此次執行開始時間

# 開始爬取
for forumName in ['meme']:
    # 抓某個看板
    print("===================")
    print(f"> 開始抓取 {forumName}")
    articles_df = fetch_forumArticles(forumName, epoch)

    fileName = f"./raw_data/{startTimeStr}_{forumName}.csv"    
    print(f"> 出檔案 {fileName}")
    articles_df.to_csv(fileName, index=False)

    print(f"> 結束抓取 {forumName}")
    print("===================")

> 開始抓取 meme
>> current: 0/30
>> current: 10/30
>> current: 20/30
> 出檔案 ./raw_data/2022-08-05_16-06_meme.csv
> 結束抓取 meme
CPU times: user 1.41 s, sys: 91.6 ms, total: 1.51 s
Wall time: 19.3 s


# 讀檔案

In [24]:
import os
import glob

# 抓到所有csv檔案的名稱
all_files = glob.glob('./raw_data/*.csv')

# 讀取所有csv
all_df = []

for f in all_files:
    _df = pd.read_csv(f)
    all_df.append(_df)

# 合併所有檔案
df = pd.concat(all_df).reset_index(drop=True)

# 抓取文章中用戶ID

In [ ]:
# Selenium模擬瀏覽器
drivePath = './chromedriver'
browser = webdriver.Chrome(drivePath)

In [ ]:
idlist = []
error_id=[]
# articles = allDf.Link.values.tolist()
articles = df.Link

for art in tqdm(articles):
    try:
        browser.get(f'https://www.dcard.tw{art}')
        html = browser.page_source
        soup = BeautifulSoup(html)
        artid = soup.find('a', class_='cPeMKB').get('href')
        print('post: ', art)
        print('artid: ', artid)
    #     artName = art.find('name', class_='elZJuv')
        usersdict = {"ID": artid}
        idlist.append(usersdict)
    except:
        print('>> Error Id: ', art)
        error_id.append(art)
        continue

  8%|██████████▊                                                                                                                      | 1/12 [00:04<00:54,  4.91s/it]

post:  /f/job/p/239620931
artid:  /@m0966n


 17%|█████████████████████▌                                                                                                           | 2/12 [00:07<00:36,  3.68s/it]

post:  /f/meme/p/239623376
artid:  /@bill1911888


 25%|████████████████████████████████▎                                                                                                | 3/12 [00:10<00:30,  3.41s/it]

...

post:  /f/meme/p/239620076
artid:  /@db200070


 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 10/12 [00:50<00:12,  6.25s/it]

post:  /f/meme/p/239620070
artid:  /@fishbone1011


 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎          | 11/12 [00:53<00:05,  5.30s/it]

post:  /f/meme/p/239620042
artid:  /@thomas1919


In [12]:
# 合併資料
df1 = pd.DataFrame(idlist)
allDf = pd.concat([df, df1], axis=1)
allDf

,Title,Link,ForumName,ID
0,想賺多一點錢應付生活費,/f/job/p/239620931,job,/@m0966n


## 整理df

In [13]:
allDf['IdList'] = idlist
allDf['Id_new'] = allDf.IdList.apply(lambda x: x['ID'])
allDf.drop(columns=['IdList'], inplace = True)
allDf['NewURL'] =  "https://www.dcard.tw" + allDf.Id_new
allDf.drop(columns=['Id_new'], inplace = True)
allDf

,Title,Link,ForumName,ID,NewURL
0,想賺多一點錢應付生活費,/f/job/p/239620931,job,/@m0966n,https://www.dcard.tw/@m0966n


# 抓取個人主頁貼文看版

In [ ]:
# Selenium模擬瀏覽器
drivePath = './chromedriver'
browser = webdriver.Chrome(drivePath)

In [14]:
personal_page = allDf.NewURL
board_dict = []

for personal_board in tqdm(personal_page):
    browser.get(personal_board)
    html = browser.page_source
    soup = BeautifulSoup(html)
# 從網址選出看板
    personal_posts = soup.select(".kfHo")
    all_forums = [a['href'].split("/")[2] for a in personal_posts]
    forumsList = list(set(all_forums))
    print(forumsList)
# 共幾種看板/共發了計次
    forumsCountDict = pd.DataFrame({"Forum": all_forums}).reset_index().groupby(
        "Forum").agg({"index": "count"}).to_dict()['index']
    userForumsDict = {"ForumsList": forumsList,
                      "ForumsCountDict": forumsCountDict}
    board_dict.append(userForumsDict)
    
userForumsDf = pd.DataFrame(board_dict)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.87s/it]

['job', 'relationship']


In [15]:
# 合併資料
final_result = pd.concat([allDf, userForumsDf], axis=1, join='inner')

In [16]:
final_result

,Title,Link,ForumName,ID,NewURL,ForumsList,ForumsCountDict
0,想賺多一點錢應付生活費,/f/job/p/239620931,job,/@m0966n,https://www.dcard.tw/@m0966n,"[job, relationship]","{'job': 1, 'relationship': 1}"


In [17]:
final_result.ForumsCountDict.keys

<bound method Series.keys of 0    {'job': 1, 'relationship': 1}
Name: ForumsCountDict, dtype: object>

In [18]:
final_result.ForumsList

0    [job, relationship]
Name: ForumsList, dtype: object